In [1]:
import numpy as np
import time
import matplotlib.pyplot as plt
from KMCLib import *

# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot

from KMCAnalysis_single_trajectory import *
from gradient_color_line import *

# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

# -----------------------------------------------------------------------------
# KMCLib version 2.0-a1
# Distributed under the GPLv3 license
# Copyright (C)  2012-2016  Mikael Leetmaa
# Developed by Mikael Leetmaa <leetmaa@kth.se>
#
# This program is distributed in the hope that it will be useful
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# LICENSE and README files, and the source code, for details.
#
# You should have received a copy of the GNU General Public License version 3
# (GPLv3) along with this program. If not, see <http://www.gnu.org/licenses/>.
# -----------------------------------------------------------------------------



In [2]:
# Define a squared unit cell.
cell_vectors = [[   1.0000000e+00,   0.000000e+00,   0.000000e+00],
                [   0.000000e+00,   1.000000e+00,   0.000000e+00],
                [   0.000000e+00,   0.000000e+00,   1.000000e+00]]
# idk what this does
basis_points = [[   0.000000e+00,   0.000000e+00,   0.000000e+00]]
unit_cell = KMCUnitCell(cell_vectors=cell_vectors,
                        basis_points=basis_points)

In [3]:
# Define the lattice
size = 100
lattice = KMCLattice(unit_cell=unit_cell,
                     repetitions=(size,size,1),
                     periodic=(True, True, False))

In [4]:
# Size of the polymer 
polymer_size = 3
if polymer_size >= size:
    print('polymer bigger than the box')
    raise InvalideSizeError
# Generate the initial configuration array
types = ['E']*(size*size)
# index of a random non-border site
yr,xr = np.random.randint(0,size-(1+polymer_size)),np.random.randint(0,size)
bound_IDs = dict()
for i in range(polymer_size):
    if i ==0:
        print(yr*size+xr)
        bound_IDs[xr+size*yr] = (xr+size*(yr+2),)
    elif i==polymer_size-1:
        bound_IDs[xr+size*(yr+2*i)] = (xr+size*(yr+2*(i-1)),)
    else:
        bound_IDs[xr+size*(yr+2*i)] = (xr+size*(yr+2*i-2),xr+size*(yr+2*(i+1)))
    #add all the neighboring particles
    types[xr+size*(yr+2*i)] = 'P'#+str(i)



config = KMCConfiguration(lattice=lattice,
                          types=types,
                          possible_types=['E'].append(['P']),
                          bound_IDs = bound_IDs)#+str(i) for i in range(polymer_size)]))
#print(config._script())

7677


In [5]:
print(config.bound_IDs)

{7677: (7877,), 7877: (7677, 8077), 8077: (7877,)}


In [6]:
from Simple_Processes import *
from Rate_Calculator import *
#processes = Generate_Processes(3)
print(processes.__len__())
interactions = KMCInteractions(processes=processes)
interactions.setRateCalculator(rate_calculator=CustomPlugin)

4


In [7]:
# Generate the KMC model to run.
model = KMCLatticeModel(configuration=config,
                        interactions=interactions)
control_parameters = KMCControlParameters(number_of_steps=10,
                                          dump_interval=1,
                                          analysis_interval=1,
                                          seed=None)
# generate the analysis object
MyAnalysis = [SingleTrajectory('P',
                              control_parameters.analysisInterval(),
                              control_parameters.numberOfSteps())]

In [8]:
t0 = time.perf_counter()
model.run(control_parameters,trajectory_filename='test.py',
          trajectory_type='xyz',
          analysis = MyAnalysis,)
print('execution time : '+str(time.perf_counter()-t0))

 KMCLib: setting up the backend C++ object.
12
<KMCLib.Backend.Backend.Interactions; proxy of <Swig Object of type 'Interactions *' at 0x7f3cc10de2d0> >
[7677 7877 8077]
 KMCLib: Runing for 10 steps, starting from time: 0.000000

 KMCLib: 1 steps executed. time:     1.0534497804e-01 
 KMCLib: 2 steps executed. time:     1.9411667296e-01 
 KMCLib: 3 steps executed. time:     2.7861039914e-01 
 KMCLib: 4 steps executed. time:     5.2514581636e-01 
 KMCLib: 5 steps executed. time:     3.9661564431e+08 
 KMCLib: 6 steps executed. time:     6.5112857993e+08 
 KMCLib: 7 steps executed. time:     6.5112857999e+08 
 KMCLib: 8 steps executed. time:     1.7001988828e+09 
 KMCLib: 9 steps executed. time:     3.4928556321e+09 
 KMCLib: 10 steps executed. time:     6.6334951207e+09 
execution time : 2.626801624000109
